<a href="https://colab.research.google.com/github/yuri-kasahara/Workshop-Tekst-som-data/blob/main/Copy_of_Example_scraper_NIBR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This is a script with an example of a scraper to obtain a list of hearings held by the Norwegian government with their respective comments.**

The two cells below will load the packages you need to run the scraper.

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.syKNfejRMS/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.pTPuVtix8l/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.yIF7WSCI4S/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [ ]:
!apt-get update
!apt-get install chromium chromium-driver
!pip3 install selenium


import regex as re
import requests
import time
import pandas as pd
import os
import logging
from bs4 import BeautifulSoup as BS
import urllib.request

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service


options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument('--disable-blink-features=AutomationControlled')

service_driver = Service(executable_path=r'/usr/bin/chromedriver')



Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Get:8 http://deb.debian.org/debian-security buster/updates InRelease [34.8 kB]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [645 kB]
Hit:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:13 http://deb.debian.org/debian buster/main amd64 Packages [10.7 MB]
H

Mount your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Now, let's get some data!

In [ ]:
 ## This is a link for one of the pages containing the list of reports published by NIBR.

url = "https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=0"

driver.get(url)
html = driver.page_source

## You can use the developer tools of your web browser to help you identify the areas you want to scrape. You will probably also have to introduce some loops to properly separate each entry you want.
soup = BS(html, "html.parser")
link_reports = soup.find_all('h4', {'class':'artifact-title'})


print(link_reports)


In [ ]:
## Let's make a list of links to cover all 569 reports from NIBR

Links = []

sequence = list(range(0, 561, 20))

for s in sequence:
    url = "https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=0"+str(s)
    Links.append(url)

print(Links)

['https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=00', 'https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=020', 'https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=040', 'https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=060', 'https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=080', 'https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=0100', 'https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=0120', 'https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=0140', 'https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=0160', 'https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=0180', 'https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=0200', 'https://oda.oslomet.no/oda-xmlui/han

In [ ]:
#Let's get for all other SVA institutes

Links_AFI = []

sequence = list(range(0, 341, 20))

for s in sequence:
    url_AFI = "https://oda.oslomet.no/oda-xmlui/handle/11250/2740305/recent-submissions?offset=0"+str(s)
    Links_AFI.append(url_AFI)

Links_NOVA = []

sequence = list(range(0, 421, 20))

for s in sequence:
    url_NOVA = "https://oda.oslomet.no/oda-xmlui/handle/11250/2740300/recent-submissions?offset=0"+str(s)
    Links_NOVA.append(url_NOVA)

Links_SIFO = []

sequence = list(range(0, 341, 20))

for s in sequence:
    url_SIFO = "https://oda.oslomet.no/oda-xmlui/handle/11250/2740281/recent-submissions?offset=0"+str(s)
    Links_SIFO.append(url_SIFO)



In [ ]:
## We have first to create empty lists to hold our scraped content

Rapport_tittel = []
Url_report = []
Institute = []

## Then we have to ask load our 'driver' and tell it what we want to collect. Let's introduce a loop so we cover all links in our previous list.
driver = webdriver.Chrome(service=service_driver, options=options)

for l in Links:

    print("Getting information from: ", l)
    driver.get(l)
    html = driver.page_source

## You can use the developer tools of your web browser to help you identify the areas you want to scrape. You will probably also have to introduce some loops to properly separate each entry you want.
    soup = BS(html, "html.parser")
    link_reports = soup.find_all('h4', {'class':'artifact-title'})

    for r in link_reports:
# These lines give you the name of the hearing
        t = r.find('a')
        print(t)
        Rapport_tittel.append(t.text.strip())
# These lines give you the link for the webpage of the hearing
        u = r.find('a')['href']
        lenken = "oda.oslomet.no"+str(u)
        Url_report.append(lenken)

        Institute.append('NIBR')

    time.sleep(3)

driver.quit()


# Then you have to put all the text collected in a dataframe format by turning the lists into variables.
Reports_list = {}
Reports_list['Rapport_tittel'] = Rapport_tittel
Reports_list['Url_report'] = Url_report
Reports_list['Institute'] = Institute


# Here you will make the dataframe
df = pd.DataFrame.from_dict(Reports_list, orient='columns')

print(df)

# Now save it on your drive or on your computer
df.to_csv('drive/MyDrive/WorkshopTSD/reports_list.csv', index=False)


Getting information from:  https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=00
<a href="/oda-xmlui/handle/11250/3111137">Bærekraftig befolkningsutvikling og grønn distriktsomstilling</a>
<a href="/oda-xmlui/handle/11250/3103781">Risikovurdering og reintegrering av radikaliserte individer i Norden</a>
<a href="/oda-xmlui/handle/11250/3085164">Midtvegsevaluering av Program for folkehelsearbeid i Vestland</a>
<a href="/oda-xmlui/handle/11250/3054721">Flyktningers møte med NAV.  Kommunikasjon og tilgjengelighet i en digital kontekst.</a>
<a href="/oda-xmlui/handle/11250/3050589">Vil gjerne vite mer: Kompetansebehov om negativ sosial kontroll og æresrelatert vold blant ansatte i opplærings- og integreringstjenestene</a>
<a href="/oda-xmlui/handle/11250/3045883">Evaluering av opptrappingsplanen mot vold og overgrep. Sluttrapport</a>
<a href="/oda-xmlui/handle/11250/3045754">Boligmarkedsanalyse: Fredrikstad mot 2035</a>
<a href="/oda-xmlui/handle/11250/3041340">

In [ ]:
## Do the same for AFI

Rapport_tittel = []
Url_report = []
Institute = []
## Then we have to ask load our 'driver' and tell it what we want to collect. Let's introduce a loop so we cover all links in our previous list.
driver = webdriver.Chrome(service=service_driver, options=options)

for l in Links_AFI:

    print("Getting information from: ", l)
    driver.get(l)
    html = driver.page_source

## You can use the developer tools of your web browser to help you identify the areas you want to scrape. You will probably also have to introduce some loops to properly separate each entry you want.
    soup = BS(html, "html.parser")
    link_reports = soup.find_all('h4', {'class':'artifact-title'})

    for r in link_reports:
# These lines give you the name of the hearing
        t = r.find('a')
        print(t)
        Rapport_tittel.append(t.text.strip())
# These lines give you the link for the webpage of the hearing
        u = r.find('a')['href']
        lenken = "oda.oslomet.no"+str(u)
        Url_report.append(lenken)

        Institute.append('AFI')

    time.sleep(3)

driver.quit()


# Then you have to put all the text collected in a dataframe format by turning the lists into variables.
Reports_list = {}
Reports_list['Rapport_tittel'] = Rapport_tittel
Reports_list['Url_report'] = Url_report
Reports_list['Institute'] = Institute


# Here you will make the dataframe
df = pd.DataFrame.from_dict(Reports_list, orient='columns')

print(df)

# Now save it on your drive or on your computer
df.to_csv('drive/MyDrive/WorkshopTSD/reports_list_AFI.csv', index=False)


Getting information from:  https://oda.oslomet.no/oda-xmlui/handle/11250/2740305/recent-submissions?offset=00
<a href="/oda-xmlui/handle/11250/3111185">Partssamarbeid i renholdsbransjen</a>
<a href="/oda-xmlui/handle/11250/3106023">Heltid, deltid og sykefravær: Litteratursøk mot nordisk offentlig sektor og innspill til videre forskning</a>
<a href="/oda-xmlui/handle/11250/3102632">Arbeids- og velferdstjenestene: En forenklet kunnskapsoversikt over arbeidsinkluderingssystemet</a>
<a href="/oda-xmlui/handle/11250/3100371">Arbeidsmiljø og arbeidshelse – en forenklet kunnskapsoversikt</a>
<a href="/oda-xmlui/handle/11250/3096673">The Norwegian version of the Copenhagen Psychosocial Questionnaire (COPSOQ III): Initial validation study using a national sample of registered nurses</a>
<a href="/oda-xmlui/handle/11250/3096668">Covid, work reorganisation and trust: the importance of employment relations</a>
<a href="/oda-xmlui/handle/11250/3090121">Vekterbransjens samfunnsbidrag i Norge</a>
<a 

In [ ]:
## Do the same for NOVA

Rapport_tittel = []
Url_report = []
Institute = []
## Then we have to ask load our 'driver' and tell it what we want to collect. Let's introduce a loop so we cover all links in our previous list.
driver = webdriver.Chrome(service=service_driver, options=options)

for l in Links_NOVA:

    print("Getting information from: ", l)
    driver.get(l)
    html = driver.page_source

## You can use the developer tools of your web browser to help you identify the areas you want to scrape. You will probably also have to introduce some loops to properly separate each entry you want.
    soup = BS(html, "html.parser")
    link_reports = soup.find_all('h4', {'class':'artifact-title'})

    for r in link_reports:
# These lines give you the name of the hearing
        t = r.find('a')
        print(t)
        Rapport_tittel.append(t.text.strip())
# These lines give you the link for the webpage of the hearing
        u = r.find('a')['href']
        lenken = "oda.oslomet.no"+str(u)
        Url_report.append(lenken)

        Institute.append('NOVA')

    time.sleep(3)

driver.quit()


# Then you have to put all the text collected in a dataframe format by turning the lists into variables.
Reports_list = {}
Reports_list['Rapport_tittel'] = Rapport_tittel
Reports_list['Url_report'] = Url_report
Reports_list['Institute'] = Institute


# Here you will make the dataframe
df = pd.DataFrame.from_dict(Reports_list, orient='columns')

print(df)

# Now save it on your drive or on your computer
df.to_csv('drive/MyDrive/WorkshopTSD/reports_list_NOVA.csv', index=False)


Getting information from:  https://oda.oslomet.no/oda-xmlui/handle/11250/2740300/recent-submissions?offset=00
<a href="/oda-xmlui/handle/11250/3106050">«Jeg er for mangfold, men…». Diskriminering av muslimer på arbeidsmarkedet: erfaringer, perspektiver og tiltak</a>
<a href="/oda-xmlui/handle/11250/3105542">Barn og unge i familier med fluktbakgrunn. Erfaringer med inkludering og tilhørighet</a>
<a href="/oda-xmlui/handle/11250/3105590">Politisk mestringstro i ungdomsskolen. Casestudie om demokrati- og medborgerskapsundervisning i fire norske ungdomsskoler</a>
<a href="/oda-xmlui/handle/11250/3105057">Nasjonal trygghetsundersøkelse 2022</a>
<a href="/oda-xmlui/handle/11250/3095162">Med døra på gløtt. Barn og unge inn i familievernet?</a>
<a href="/oda-xmlui/handle/11250/3091723">Er koordinatorordningen for familier med funksjonshemmede barn bærekraftig?</a>
<a href="/oda-xmlui/handle/11250/3090376">Eldres boligønsker og bosituasjon. En undersøkelse blant hjemmeboende over 75 år</a>
<a h

In [ ]:
## Do the same for SIFO

Rapport_tittel = []
Url_report = []
Institute = []
## Then we have to ask load our 'driver' and tell it what we want to collect. Let's introduce a loop so we cover all links in our previous list.
driver = webdriver.Chrome(service=service_driver, options=options)

for l in Links_SIFO:

    print("Getting information from: ", l)
    driver.get(l)
    html = driver.page_source

## You can use the developer tools of your web browser to help you identify the areas you want to scrape. You will probably also have to introduce some loops to properly separate each entry you want.
    soup = BS(html, "html.parser")
    link_reports = soup.find_all('h4', {'class':'artifact-title'})

    for r in link_reports:
# These lines give you the name of the hearing
        t = r.find('a')
        print(t)
        Rapport_tittel.append(t.text.strip())
# These lines give you the link for the webpage of the hearing
        u = r.find('a')['href']
        lenken = "oda.oslomet.no"+str(u)
        Url_report.append(lenken)

        Institute.append('SIFO')

    time.sleep(3)

driver.quit()


# Then you have to put all the text collected in a dataframe format by turning the lists into variables.
Reports_list = {}
Reports_list['Rapport_tittel'] = Rapport_tittel
Reports_list['Url_report'] = Url_report
Reports_list['Institute'] = Institute


# Here you will make the dataframe
df = pd.DataFrame.from_dict(Reports_list, orient='columns')

print(df)

# Now save it on your drive or on your computer
df.to_csv('drive/MyDrive/WorkshopTSD/reports_list_SIFO.csv', index=False)


Getting information from:  https://oda.oslomet.no/oda-xmlui/handle/11250/2740281/recent-submissions?offset=00
<a href="/oda-xmlui/handle/11250/3107927">Eksponering for markedsføring av usunn mat og drikke. Reklame rettet mot barn og unge i sosiale medier</a>
<a href="/oda-xmlui/handle/11250/3101053">Barns forbruk i videospill og hvordan det påvirker sosiale relasjoner. Delrapport 2 fra prosjektet «Pay to play»</a>
<a href="/oda-xmlui/handle/11250/3101047">Kartlegging av manipulerende spilldesign. Delrapport 1 fra prosjektet «Pay to play»</a>
<a href="/oda-xmlui/handle/11250/3097051">Barns digitale forbrukervern. En kunnskapsgjennomgang om digital sårbarhet og negative konsekvenser ved kommersiell påvirkning</a>
<a href="/oda-xmlui/handle/11250/3094312">Mapping of the digital climate nudges in Nordic online grocery stores</a>
<a href="/oda-xmlui/handle/11250/3091870">Dyrtid 4: Det er ikke over ennå. Husholdenes økonomiske trygghet i august 2023</a>
<a href="/oda-xmlui/handle/11250/30895

In [ ]:
df_NIBR = pd.read_csv("drive/MyDrive/WorkshopTSD/reports_list.csv", sep=',', dtype='string')


pd.options.display.max_colwidth = 150

print(df_NIBR)

                                                                                                                                 Rapport_tittel  \
0                                                                                 Bærekraftig befolkningsutvikling og grønn distriktsomstilling   
1                                                                          Risikovurdering og reintegrering av radikaliserte individer i Norden   
2                                                                                 Midtvegsevaluering av Program for folkehelsearbeid i Vestland   
3                                                           Flyktningers møte med NAV.  Kommunikasjon og tilgjengelighet i en digital kontekst.   
4    Vil gjerne vite mer: Kompetansebehov om negativ sosial kontroll og æresrelatert vold blant ansatte i opplærings- og integreringstjenestene   
..                                                                                                                    

In [ ]:
URLs_reports= df_NIBR['Url_report']
print(URLs_reports)

0          oda.oslomet.no/oda-xmlui/handle/11250/3111137
1          oda.oslomet.no/oda-xmlui/handle/11250/3103781
2          oda.oslomet.no/oda-xmlui/handle/11250/3085164
3          oda.oslomet.no/oda-xmlui/handle/11250/3054721
4          oda.oslomet.no/oda-xmlui/handle/11250/3050589
                             ...                        
565    oda.oslomet.no/oda-xmlui/handle/20.500.12199/1282
566    oda.oslomet.no/oda-xmlui/handle/20.500.12199/1281
567    oda.oslomet.no/oda-xmlui/handle/20.500.12199/1280
568    oda.oslomet.no/oda-xmlui/handle/20.500.12199/1279
569    oda.oslomet.no/oda-xmlui/handle/20.500.12199/1268
Name: Url_report, Length: 570, dtype: string


Now, let's get data on the individual contents of each report.

The scraper below will use the links from the previous dataset.

In [ ]:
Title = []
Abstract = []
Year = []
Institute = []

driver = webdriver.Chrome(service=service_driver, options=options)

for link in URLs_reports:
    print("Getting information from: ", link)
    driver.get("https://" + str(link))
    time.sleep(3)
    html = driver.page_source
    soup = BS(html, "html.parser")

    t = soup.find('h2', {'class':'page-header first-page-header'})
    Title.append(t.text.strip())
    Institute.append('NIBR')

    try:
        abs = soup.find_all('div', {'class':'simple-item-view-description item-page-field-wrapper table'})
        pattern_found = False
        for a in abs:
            clean_a = a.text.strip()
            if clean_a.startswith("Sammendrag") or clean_a.startswith("Abstract"):
              clean_abs = re.sub(r'^(Sammendrag|Abstract)\n', '', clean_a)
              print(clean_abs)
              Abstract.append(clean_abs)
              pattern_found = True

        if not pattern_found:
            Abstract.append(None)
    except:
        Abstract.append(None)

    try:
        y = soup.find('div', {'class':'simple-item-view-date word-break item-page-field-wrapper table'})
        clean_y = y.text.strip()
        clean_y = re.sub(r'^Utgivelsesdato', '', clean_y)
        pattern = r'\b\d{4}\b'
        clean_y = re.search(pattern, clean_y)
        Year.append(clean_y.group())

    except:
        Year.append(None)
driver.quit()

Info_reports = {}

Info_reports['Title'] = Title
Info_reports['Abstract'] = Abstract
Info_reports['Year'] = Year
Info_reports['Institute'] = Institute


df3 = pd.DataFrame.from_dict(Info_reports, orient='columns')

df3.to_csv('drive/MyDrive/WorkshopTSD/Info_reports.csv', index=False)

print(df3)


Getting information from:  oda.oslomet.no/oda-xmlui/handle/11250/3111137
Rapporten handler om bærekraftig befolkningsutvikling i distriktskommuner og hvilke faktorer for attraktivitet og kompetanse som kan bidra til grønn omstilling. Rapporten beskriver indikatorer som distriktskommuner kan benytte for å følge og overvåke samfunnsutvikling på disse områdene. Det empiriske grunnlaget er kommunene i Lofoten, Harstad og Narvik.
Getting information from:  oda.oslomet.no/oda-xmlui/handle/11250/3103781
Rapporten belyser arbeidet med risikovurdering og reintegrering av radikaliserte individer i de fire nordiske landene, Danmark, Sverige, Norge og Finland. Vi har intervjuet praktikere og andre myndighetspersoner med særlig erfaring med denne typen arbeid i Norden, samt gjennomgått forskning og annen litteratur. På bakgrunn av dette belyser vi sentrale praksiser og tiltak innen risikovurderinger og reintegrering, samt vurderinger av hvordan disse praksisene og tiltakene fungerer. Vi peker på se

In [ ]:
df_AFI = pd.read_csv("drive/MyDrive/WorkshopTSD/reports_list_AFI.csv", sep=',', dtype='string')
URLs_reports_AFI = df_AFI['Url_report']

df_NOVA = pd.read_csv("drive/MyDrive/WorkshopTSD/reports_list_NOVA.csv", sep=',', dtype='string')
URLs_reports_NOVA = df_NOVA['Url_report']

df_SIFO = pd.read_csv("drive/MyDrive/WorkshopTSD/reports_list_SIFO.csv", sep=',', dtype='string')
URLs_reports_SIFO = df_SIFO['Url_report']

In [ ]:
Title = []
Abstract = []
Year = []
Institute = []

driver = webdriver.Chrome(service=service_driver, options=options)

for link in URLs_reports_AFI:
    print("Getting information from: ", link)
    driver.get("https://" + str(link))
    time.sleep(3)
    html = driver.page_source
    soup = BS(html, "html.parser")

    t = soup.find('h2', {'class':'page-header first-page-header'})
    Title.append(t.text.strip())
    Institute.append('AFI')

    try:
        abs = soup.find_all('div', {'class':'simple-item-view-description item-page-field-wrapper table'})
        pattern_found = False
        for a in abs:
            clean_a = a.text.strip()
            if clean_a.startswith("Sammendrag") or clean_a.startswith("Abstract"):
              clean_abs = re.sub(r'^(Sammendrag|Abstract)\n', '', clean_a)
              print(clean_abs)
              Abstract.append(clean_abs)
              pattern_found = True

        if not pattern_found:
            Abstract.append(None)
    except:
        Abstract.append(None)

    try:
        y = soup.find('div', {'class':'simple-item-view-date word-break item-page-field-wrapper table'})
        clean_y = y.text.strip()
        clean_y = re.sub(r'^Utgivelsesdato', '', clean_y)
        pattern = r'\b\d{4}\b'
        clean_y = re.search(pattern, clean_y)
        Year.append(clean_y.group())

    except:
        Year.append(None)
driver.quit()

Info_reports = {}

Info_reports['Title'] = Title
Info_reports['Abstract'] = Abstract
Info_reports['Year'] = Year
Info_reports['Institute'] = Institute

df3 = pd.DataFrame.from_dict(Info_reports, orient='columns')

df3.to_csv('drive/MyDrive/WorkshopTSD/Info_reports_AFI.csv', index=False)

print(df3)


Getting information from:  oda.oslomet.no/oda-xmlui/handle/11250/3111185
Denne rapporten presenterer nå-situasjonen for partssamarbeidet i de ti virksomhetene som deltar i prosjektet Ren Tilstedeværelse. Rapporten innledes med en beskrivelse av sentrale trekk ved den norske arbeidslivsmodellen og relevante funn fra tidligere forskning. Datagrunnlaget for rapporten er observasjoner fra samlinger, intervjuer av ledere og en spørreundersøkelse blant mellomledere. Basert på dataene som er samlet inn hittil i prosjektet finner vi samstemthet på noen områder og sprik på andre når det kommer til partenes oppfatninger om dagens partssamarbeid. Både de tillitsvalgte og lederne uttrykker et behov for å få på plass flere tillitsvalgte og å jobbe videre med mangfoldsledelse. Et område hvor oppfatningene spriker er når det gjelder mellomledernes kunnskap om lov- og avtaleverk. Her er de tillitsvalgte og toppledelsen enige om at det finnes forbedringspotensial, mens mellomlederne vurderer sin kunnsk

In [ ]:
Title = []
Abstract = []
Year = []
Institute = []

driver = webdriver.Chrome(service=service_driver, options=options)

for link in URLs_reports_NOVA:
    print("Getting information from: ", link)
    driver.get("https://" + str(link))
    time.sleep(3)
    html = driver.page_source
    soup = BS(html, "html.parser")

    t = soup.find('h2', {'class':'page-header first-page-header'})
    Title.append(t.text.strip())
    Institute.append('NOVA')

    try:
        abs = soup.find_all('div', {'class':'simple-item-view-description item-page-field-wrapper table'})
        pattern_found = False
        for a in abs:
            clean_a = a.text.strip()
            if clean_a.startswith("Sammendrag") or clean_a.startswith("Abstract"):
              clean_abs = re.sub(r'^(Sammendrag|Abstract)\n', '', clean_a)
              print(clean_abs)
              Abstract.append(clean_abs)
              pattern_found = True

        if not pattern_found:
            Abstract.append(None)
    except:
        Abstract.append(None)

    try:
        y = soup.find('div', {'class':'simple-item-view-date word-break item-page-field-wrapper table'})
        clean_y = y.text.strip()
        clean_y = re.sub(r'^Utgivelsesdato', '', clean_y)
        pattern = r'\b\d{4}\b'
        clean_y = re.search(pattern, clean_y)
        Year.append(clean_y.group())

    except:
        Year.append(None)
driver.quit()

Info_reports = {}

Info_reports['Title'] = Title
Info_reports['Abstract'] = Abstract
Info_reports['Year'] = Year
Info_reports['Institute'] = Institute

df3 = pd.DataFrame.from_dict(Info_reports, orient='columns')

df3.to_csv('drive/MyDrive/WorkshopTSD/Info_reports_NOVA.csv', index=False)

print(df3)


Getting information from:  oda.oslomet.no/oda-xmlui/handle/11250/3106050
Tema for denne rapporten er diskriminering av muslimer og personer som antas å være muslimer. Rapporten har to hovedmål: Først å kartlegge omfanget, innholdet og konsekvensene av denne diskrimineringen. Deretter å identifisere årsakene til slik diskriminering for å kunne foreslå effektive tiltak for å redusere problemet.
Rapporten er resultatet av prosjektet «Diskriminering av muslimer i arbeidslivet». Prosjektet er gjennomført på oppdrag fra Arbeids- og inkluderingsdepartementet og er et samarbeid mellom Velferdsforskningsinstituttet NOVA og Oslo Economics.
Getting information from:  oda.oslomet.no/oda-xmlui/handle/11250/3105542
Rapporten belyser hvilke erfaringer barn og unge med fluktbakgrunn har med inkludering i Norge. 
Vi finner at barna har behov for mer støtte i bosettingsfasen, særlig knyttet til fritidsaktiviteter. Skolen er en viktig arena for inkludering, men måten undervisningen er organisert på, kan 

In [ ]:
Title = []
Abstract = []
Year = []
Institute = []

driver = webdriver.Chrome(service=service_driver, options=options)

for link in URLs_reports_SIFO:
    print("Getting information from: ", link)
    driver.get("https://" + str(link))
    time.sleep(3)
    html = driver.page_source
    soup = BS(html, "html.parser")

    t = soup.find('h2', {'class':'page-header first-page-header'})
    Title.append(t.text.strip())
    Institute.append('SIFO')

    try:
        abs = soup.find_all('div', {'class':'simple-item-view-description item-page-field-wrapper table'})
        pattern_found = False
        for a in abs:
            clean_a = a.text.strip()
            if clean_a.startswith("Sammendrag") or clean_a.startswith("Abstract"):
              clean_abs = re.sub(r'^(Sammendrag|Abstract)\n', '', clean_a)
              print(clean_abs)
              Abstract.append(clean_abs)
              pattern_found = True

        if not pattern_found:
            Abstract.append(None)
    except:
        Abstract.append(None)

    try:
        y = soup.find('div', {'class':'simple-item-view-date word-break item-page-field-wrapper table'})
        clean_y = y.text.strip()
        clean_y = re.sub(r'^Utgivelsesdato', '', clean_y)
        pattern = r'\b\d{4}\b'
        clean_y = re.search(pattern, clean_y)
        Year.append(clean_y.group())

    except:
        Year.append(None)
driver.quit()

Info_reports = {}

Info_reports['Title'] = Title
Info_reports['Abstract'] = Abstract
Info_reports['Year'] = Year
Info_reports['Institute'] = Institute

df3 = pd.DataFrame.from_dict(Info_reports, orient='columns')

df3.to_csv('drive/MyDrive/WorkshopTSD/Info_reports_SIFO.csv', index=False)

print(df3)


Getting information from:  oda.oslomet.no/oda-xmlui/handle/11250/3107927
Nylig ble det flertall på Stortinget for et forbud mot markedsføring av usunn mat og drikke rettet mot barn og unge, samt forhøyet aldersgrense for kjøp av energidrikk. Forskning viser at markedsføring av mat og drikke med høyt innhold av salt, sukker og fett har negativ innvirkning på barn og unges kostholdsvaner. Det har også vært en betydelig økning i inntaket av energidrikker blant unge. Denne rapporten analyserer sosiale medier-innlegg fra populære influensere, med fokus på reklame for usunn mat og drikke, samt markedsføringsstrategier til selskapene som Monster, Red Bull og PRIME. Omtrent 24 prosent av innleggene samlet fra influensere var reklame, med mat- og drikkeannonser som den største kategorien, dominert av sports- og energidrikker. Monster, Red Bull og PRIME ser ut til å bygge en merkevare rundt sport og idrettsutøvere. Monster og Red Bull ser ut til å markere seg godt i dagligvarebutikker og andre u